In [10]:
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random

from shapely.geometry import Polygon, box
from ultralytics import YOLO
from glob import glob
from sklearn.cluster import DBSCAN

In [17]:
# data_dir = glob('C:/Users/ghkdr/Desktop/성현폴더/충북대/논문/데이터/혼합검증용/data/*/*.jpg')
data_dir = glob(r'C:\Users\ghkdr\Desktop\성현폴더\충북대\논문\데이터\CH03_data2\image_origin\*.jpg')
model_dir = r'C:\Users\ghkdr\Desktop\성현폴더\충북대\논문\실험\runs\yolov9c_aug\train2\weights\best.pt'

save_dir = f"result/output{len(glob('result/*'))}"
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

roi = box(5, 5, 950,955)
reference_point = (640, 480)
model = YOLO(model_dir)
color_list = [(255, 0, 0,), (0, 255, 0), (0, 0, 255)]

for data in data_dir:
    image = cv2.imread(data)
    image_to_save = image.copy()
    cv2.rectangle(image_to_save, (5, 5), (950, 955), (255, 255, 255), 3)
    results = model(image, conf=0.8, iou=0.6, verbose=False, save=False)

    for result in results:
        area_list = []
        bbox_list = []
        for detection in result.boxes.xywh.cpu().numpy():
            x, y, w, h = detection
            bbox = box(x - w / 2, y - h / 2, x + w / 2, y + h / 2)
            
            if roi.contains(bbox):
                aspect_ratio = max(w / h, h / w)

                if aspect_ratio <= 1.2:
                    # 거리 계산
                    vector_distance = (x - reference_point[0], y - reference_point[1])
                    scalar_distance = int(np.sqrt(vector_distance[0]**2 + vector_distance[1]**2))
                    
                    # 같은 (x, y) 좌표일 경우 면적을 리스트에 추가
                    epsilon_ratio = min(((1.13 - aspect_ratio) * (w * h)) * 0.3, 0)
                    epsilon_distance = (np.log1p(np.abs(scalar_distance) / 800) * (w * h)) * 0.03
                    area = int(w * h) + epsilon_ratio - epsilon_distance
                    
                    area_list.append(area)
                    bbox_list.append((detection))
                    
    # DBSCAN 클러스터링 적용
    if len(area_list) != 0:
        area_array = np.array(area_list).reshape(-1, 1)
        eps = int(np.mean(area_list) * 0.25)
        dbscan = DBSCAN(eps=eps, min_samples=1).fit(area_array)
        
        dbscan.labels_

        for label, bbox, area in zip(dbscan.labels_, bbox_list, area_list):
            x, y, w, h = bbox
            x1 = int(x - w / 2)
            y1 = int(y - h / 2)
            x2 = int(x + w / 2)
            y2 = int(y + h / 2)
            cv2.rectangle(image_to_save, (x1, y1), (x2, y2), color_list[label], 2)
        
            text = str(int(area))  # 여기에 표시할 텍스트
            font = cv2.FONT_HERSHEY_SIMPLEX
            font_scale = 1.5
            font_color = (255, 0, 255) 
            thickness = 2
            text_position = (x2 - 20, y1 - 10) 
        
            # 텍스트를 이미지에 추가
            cv2.putText(image_to_save, text, text_position, font, font_scale, font_color, thickness)

    if len(np.unique(dbscan.labels_)) > 1:
        file_name = data.split('\\')[-1]
        cv2.imwrite(f"{save_dir}/result_{file_name}", image_to_save)

KeyboardInterrupt: 

In [18]:
# data_dir = glob('C:/Users/ghkdr/Desktop/성현폴더/충북대/논문/데이터/혼합검증용/data/*/*.jpg')
data_dir = glob(r'C:\Users\ghkdr\Desktop\성현폴더\충북대\논문\데이터\CH03_data2\image_origin\*.jpg')
model_dir = r'C:\Users\ghkdr\Desktop\성현폴더\충북대\논문\실험\runs\yolov9c_aug\train2\weights\best.pt'

save_dir = f"result/output{len(glob('result/*'))}"
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

roi = box(5, 5, 950,955)
reference_point = (640, 480)
model = YOLO(model_dir)
color_list = [(255, 0, 0,), (0, 255, 0), (0, 0, 255)]

for data in data_dir:
    image = cv2.imread(data)
    image_rgb = image.copy()
    image_to_save = image.copy()
    cv2.rectangle(image_to_save, (5, 5), (950, 955), (255, 255, 255), 3)
    results = model(image, conf=0.8, iou=0.6, verbose=False, save=False)

    for result in results:
        area_list = []
        bbox_list = []
        for detection in result.boxes.xywh.cpu().numpy():
            x, y, w, h = detection
            bbox = box(x - w / 2, y - h / 2, x + w / 2, y + h / 2)
            
            if roi.contains(bbox):
                aspect_ratio = max(w / h, h / w)

                if aspect_ratio <= 1.2:
                    # xywh를 xyxy로 변환
                    x1, y1 = int(x - w // 2), int(y - h // 2)
                    x2, y2 = int(x1 + w), int(y1 + h)

                    # 이미지 크롭
                    cropped_image = image_rgb[y1:y2, x1:x2].copy()

                    # 가로세로 비율 계산
                    aspect_ratio_max = max((x2 - x1) / (y2 - y1), (y2 - y1) / (x2 - x1))
                    
                    # 가우시안 블러 적용
                    blurred_image = cv2.GaussianBlur(cropped_image, (11, 11), 0)
                    
                    # 히스토그램 평탄화 적용
                    cropped_image_gray = cv2.cvtColor(blurred_image, cv2.COLOR_RGB2GRAY)
                    equalized_image = cv2.equalizeHist(cropped_image_gray)
                    
                    # 엣지 검출
                    edges = cv2.Canny(equalized_image, 50, 150)
                    
                    # 허프 원 변환을 사용한 원 검출
                    circles = cv2.HoughCircles(edges, cv2.HOUGH_GRADIENT, dp=1.2, minDist=30, param1=190, param2=70, minRadius=30, maxRadius=5000)
                    contour_image = cv2.cvtColor(equalized_image, cv2.COLOR_GRAY2BGR)
            
                    if circles is not None:
                        # 거리 계산
                        vector_distance = (x - reference_point[0], y - reference_point[1])
                        scalar_distance = int(np.sqrt(vector_distance[0]**2 + vector_distance[1]**2))
                        
                        # 같은 (x, y) 좌표일 경우 면적을 리스트에 추가
                        epsilon_ratio = min(((1.13 - aspect_ratio) * (w * h)) * 0.3, 0)
                        epsilon_distance = (np.log1p(np.abs(scalar_distance) / 800) * (w * h)) * 0.03
                        area = int(w * h) + epsilon_ratio - epsilon_distance
                        
                        area_list.append(area)
                        bbox_list.append((detection))
                    
    # DBSCAN 클러스터링 적용
    if len(area_list) != 0:
        area_array = np.array(area_list).reshape(-1, 1)
        eps = int(np.mean(area_list) * 0.25)
        dbscan = DBSCAN(eps=eps, min_samples=1).fit(area_array)
        
        dbscan.labels_

        for label, bbox, area in zip(dbscan.labels_, bbox_list, area_list):
            x, y, w, h = bbox
            x1 = int(x - w / 2)
            y1 = int(y - h / 2)
            x2 = int(x + w / 2)
            y2 = int(y + h / 2)
            cv2.rectangle(image_to_save, (x1, y1), (x2, y2), color_list[label], 2)
        
            text = str(int(area))  # 여기에 표시할 텍스트
            font = cv2.FONT_HERSHEY_SIMPLEX
            font_scale = 1.5
            font_color = (255, 0, 255) 
            thickness = 2
            text_position = (x2 - 20, y1 - 10) 
        
            # 텍스트를 이미지에 추가
            cv2.putText(image_to_save, text, text_position, font, font_scale, font_color, thickness)

    if len(np.unique(dbscan.labels_)) > 1:
        file_name = data.split('\\')[-1]
        cv2.imwrite(f"{save_dir}/result_{file_name}", image_to_save)